In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from MyEarlyStopping import MyEarlyStopping
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from keras.optimizers import adam

import timeit
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  

# load dataset 

In [3]:
train_number = 5
train_image = 30 #10:20%, 20: 40%, 30:60%

## Training dataset (40%)

In [4]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/train',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 1714 images belonging to 10 classes.


In [5]:
image_names = train_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = train_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=train_number)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [6]:
All_Inputs = [next(train_generator)[0][0] for _ in range(len(train_generator))]
All_Inputs = np.array(All_Inputs)

In [7]:
select_indexs_train = image_no<train_image
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train]
Train_participant_uniform = Train_participant_class*0+1/train_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10
#sum(Train_command_class)

## Validation dataset (20%)

In [8]:
val_data = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/validation',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 543 images belonging to 10 classes.


In [9]:
image_names = val_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
participant_class = np.array(list(map(int, participant_class)))
command_class = val_generator.classes
Val_participant_class = tf.keras.utils.to_categorical(participant_class-1, num_classes=train_number)
Val_participant_uniform = Val_participant_class*0+1/train_number
Val_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
Val_command_uniform = Val_command_class*0+1/10

In [10]:
Val_Inputs = [next(val_generator)[0][0] for _ in range(len(val_generator))]
Val_Inputs = np.array(Val_Inputs)

## Test dataset (20%)

In [11]:
test_data = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 543 images belonging to 10 classes.


In [12]:
image_names = test_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
test_unit_participant_class = np.array(list(map(int, participant_class)))
test_unit_command_class = test_generator.classes
Test_participant_class = tf.keras.utils.to_categorical(test_unit_participant_class-1, num_classes=train_number)
Test_participant_uniform = Test_participant_class*0+1/train_number
Test_command_class = tf.keras.utils.to_categorical(test_unit_command_class, num_classes=10)
Test_command_uniform = Test_command_class*0+1/10

In [13]:
Test_Inputs = [next(test_generator)[0][0] for _ in range(len(test_generator))]
Test_Inputs = np.array(Test_Inputs)

In [14]:
Test_Inputs_1 = Test_Inputs[np.where(test_unit_participant_class == 1)]
Test_command_class_1 = Test_command_class[np.where(test_unit_participant_class == 1)]
Test_Inputs_2 = Test_Inputs[np.where(test_unit_participant_class == 2)]
Test_command_class_2 = Test_command_class[np.where(test_unit_participant_class == 2)]
Test_Inputs_3 = Test_Inputs[np.where(test_unit_participant_class == 3)]
Test_command_class_3 = Test_command_class[np.where(test_unit_participant_class == 3)]
Test_Inputs_4 = Test_Inputs[np.where(test_unit_participant_class == 4)]
Test_command_class_4 = Test_command_class[np.where(test_unit_participant_class == 4)]
Test_Inputs_5 = Test_Inputs[np.where(test_unit_participant_class == 5)]
Test_command_class_5 = Test_command_class[np.where(test_unit_participant_class == 5)]

## Speaker 6 Test dataset 

In [15]:
test_data_6 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_6 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_6_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 100 images belonging to 10 classes.


In [16]:
image_names = test_generator_6.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_6 = le.fit_transform(command_class)
Test_command_class_6 = tf.keras.utils.to_categorical(test_unit_command_class_6, num_classes=10)

In [17]:
Test_Inputs_6 = [next(test_generator_6)[0][0] for _ in range(len(test_generator_6))]
Test_Inputs_6 = np.array(Test_Inputs_6)

## Speaker 7 Test dataset 

In [18]:
test_data_7 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_7 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_7_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 125 images belonging to 10 classes.


In [19]:
image_names = test_generator_7.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_7 = le.fit_transform(command_class)
Test_command_class_7 = tf.keras.utils.to_categorical(test_unit_command_class_7, num_classes=10)

In [20]:
Test_Inputs_7 = [next(test_generator_7)[0][0] for _ in range(len(test_generator_7))]
Test_Inputs_7 = np.array(Test_Inputs_7)

## Speaker 8 Test dataset 

In [21]:
test_data_8 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_8 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_8_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 101 images belonging to 10 classes.


In [22]:
image_names = test_generator_8.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_8 = le.fit_transform(command_class)
Test_command_class_8 = tf.keras.utils.to_categorical(test_unit_command_class_8, num_classes=10)

In [23]:
Test_Inputs_8 = [next(test_generator_8)[0][0] for _ in range(len(test_generator_8))]
Test_Inputs_8 = np.array(Test_Inputs_8)

# pd file store performance

In [24]:
# pd file store performance
# Perfomance = pd.DataFrame(columns = ['Model','Time','Partcp_Acc_p15','Command_Acc_p15','Partcp_Acc_p1','Command_Acc_p1',
#                                       'Partcp_Acc_p2','Command_Acc_p2','Partcp_Acc_p3','Command_Acc_p3',
#                                       'Partcp_Acc_p4','Command_Acc_p4','Partcp_Acc_p5','Command_Acc_p5','Acc_p6','Acc_p7','Acc_p8'])

Perfomance = pd.read_csv('Performance_0608_training_Data_Size.csv')
# Perfomance
Perfomance                                                        

,Model,Size,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8
0,Group,20%,47.540156,0.9724,0.9484,0.991,0.9550,0.9259,0.8796,0.9652,0.9565,0.9817,0.9633,1.00,0.99,0.67,0.528,0.3366
1,Group,20%,48.307249,0.9779,0.9521,1.000,0.9369,0.9352,0.8796,0.9739,0.9652,0.9908,0.9908,0.99,0.99,0.59,0.528,0.2970
2,Group,20%,46.201615,0.9613,0.9448,0.991,0.9369,0.9074,0.8611,0.9826,0.9652,0.9358,0.9725,0.99,0.99,0.67,0.432,0.2673
3,Group,20%,43.692362,0.9650,0.9521,1.000,0.9459,0.9074,0.8704,0.9826,0.9652,0.9358,0.9908,1.00,0.99,0.62,0.504,0.2673
4,Group,20%,44.504580,0.9687,0.9484,0.991,0.9550,0.9259,0.8796,0.9739,0.9652,0.9541,0.9541,1.00,0.99,0.59,0.472,0.3267
5,Group,20%,44.229166,0.9705,0.9521,1.000,0.9550,0.9074,0.8796,0.9826,0.9739,0.9633,0.9633,1.00,0.99,0.62,0.456,0.3069
6,Group,20%,44.535541,0.9632,0.9540,0.991,0.9369,0.9259,0.8981,0.9565,0.9652,0.9541,0.9817,0.99,0.99,0.67,0.496,0.3267
7,Group,20%,42.948731,0.9724,0.9595,1.000,0.9550,0.9074,0.8889,0.9913,0.9739,0.9633,0.9908,1.00,0.99,0.67,0.496,0.2574
8,Group,20%,45.444911,0.9724,0.9503,0.991,0.9459,0.9167,0.8796,0.9913,0.9652,0.9633,0.9725,1.00,0.99,0.60,0.552,0.2673
9,Group,20%,42.707019,0.9669,0.9540,0.991,0.9459,0.9167,0.8704,0.9739,0.9652,0.9541,0.9908,1.00,1.00,0.66,0.480,0.4059


# Create model 

In [25]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


## Group model 

In [26]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(resnet_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(train_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    resnet_model = tf.keras.Model(resnet_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])


    # resnet_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + resnet_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + resnet_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs, 
                           {resnet_model.layers[-2].name:Train_participant_class, 
                            resnet_model.layers[-1].name:Train_command_class,
                            resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                            resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                            validation_data=(Val_Inputs,
                                             {resnet_model.layers[-2].name:Val_participant_class,
                                              resnet_model.layers[-1].name:Val_command_class,
                                              resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                              resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = test_unit_participant_class
    acc_p15_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)

    predictions = resnet_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_1)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([1]*len(predicted_classes))
    acc_p1_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_2)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([2]*len(predicted_classes))
    acc_p2_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_3)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([3]*len(predicted_classes))
    acc_p3_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_4)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([4]*len(predicted_classes))
    acc_p4_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_5)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([5]*len(predicted_classes))
    acc_p5_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Group",'Size':'60%','Time':run_time,
                                    'Partcp_Acc_p15':acc_p15_s,'Command_Acc_p15':acc_p15_c,'Partcp_Acc_p1':acc_p1_s,
                                    'Command_Acc_p1':acc_p1_c,'Partcp_Acc_p2':acc_p2_s,'Command_Acc_p2':acc_p2_c,
                                    'Partcp_Acc_p3':acc_p3_s,'Command_Acc_p3':acc_p3_c,'Partcp_Acc_p4':acc_p4_s,
                                    'Command_Acc_p4':acc_p4_c,'Partcp_Acc_p5':acc_p5_s,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c + acc_p15_s:
        resnet_model.save('Initial_group_model_60p_0608.h5')
        high_acc = acc_p15_c + acc_p15_s
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('60p_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('60p_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
24/24 [==============================] - 6s 231ms/step - loss: 3.3345 - participant_output_loss: 1.1946 - command_output_loss: 2.1360 - command_output_1_loss: 8.8785e-04 - participant_output_1_loss: 0.0030 - participant_output_accuracy: 0.6107 - command_output_accuracy: 0.4127 - command_output_1_accuracy: 0.1365 - participant_output_1_accuracy: 0.1692 - val_loss: 2.7598 - val_participant_output_loss: 0.8086 - val_command_output_loss: 1.9508 - val_command_output_1_loss: 3.9016e-04 - val_participant_output_1_loss: 6.3677e-05 - val_participant_output_accuracy: 0.8508 - val_command_output_accuracy: 0.6777 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.2192
Epoch 2/10
24/24 [==============================] - 4s 165ms/step - loss: 2.4697 - participant_output_loss: 0.6195 - command_output_loss: 1.8500 - command_output_1_loss: 1.6620e-04 - participant_output_1_loss: 5.2210e-05 - participant_output_accuracy: 0.9084 - command_output_accuracy: 0.8094

Epoch 3/10
24/24 [==============================] - 3s 145ms/step - loss: 0.3347 - participant_output_loss: 0.0395 - command_output_loss: 0.2952 - command_output_1_loss: 2.5499e-05 - participant_output_1_loss: 1.9334e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9993 - command_output_1_accuracy: 0.0569 - participant_output_1_accuracy: 0.2381 - val_loss: 0.3980 - val_participant_output_loss: 0.0799 - val_command_output_loss: 0.3181 - val_command_output_1_loss: 1.8560e-05 - val_participant_output_1_loss: 2.3351e-05 - val_participant_output_accuracy: 0.9853 - val_command_output_accuracy: 0.9853 - val_command_output_1_accuracy: 0.4862 - val_participant_output_1_accuracy: 0.2302
Epoch 4/10
24/24 [==============================] - 4s 163ms/step - loss: 0.2819 - participant_output_loss: 0.0358 - command_output_loss: 0.2461 - command_output_1_loss: 1.7049e-05 - participant_output_1_loss: 1.7563e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9993

Epoch 6/10
24/24 [==============================] - 4s 163ms/step - loss: 1.6346 - participant_output_loss: 0.2167 - command_output_loss: 1.4178 - command_output_1_loss: 5.6779e-05 - participant_output_1_loss: 2.4713e-05 - participant_output_accuracy: 0.9953 - command_output_accuracy: 0.9853 - command_output_1_accuracy: 0.0575 - participant_output_1_accuracy: 0.1799 - val_loss: 1.6024 - val_participant_output_loss: 0.2381 - val_command_output_loss: 1.3642 - val_command_output_1_loss: 4.5399e-05 - val_participant_output_1_loss: 2.4649e-05 - val_participant_output_accuracy: 0.9797 - val_command_output_accuracy: 0.9705 - val_command_output_1_accuracy: 0.5028 - val_participant_output_1_accuracy: 0.1602
Epoch 7/10
24/24 [==============================] - 4s 161ms/step - loss: 1.4474 - participant_output_loss: 0.1753 - command_output_loss: 1.2720 - command_output_1_loss: 6.1507e-05 - participant_output_1_loss: 2.3533e-05 - participant_output_accuracy: 0.9973 - command_output_accuracy: 0.9906

Epoch 8/10
24/24 [==============================] - 3s 128ms/step - loss: 0.2575 - participant_output_loss: 0.0402 - command_output_loss: 0.2173 - command_output_1_loss: 1.5175e-05 - participant_output_1_loss: 1.6910e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9993 - command_output_1_accuracy: 0.0334 - participant_output_1_accuracy: 0.1926 - val_loss: 0.3290 - val_participant_output_loss: 0.0781 - val_command_output_loss: 0.2509 - val_command_output_1_loss: 1.6532e-05 - val_participant_output_1_loss: 2.2314e-05 - val_participant_output_accuracy: 0.9926 - val_command_output_accuracy: 0.9834 - val_command_output_1_accuracy: 0.0074 - val_participant_output_1_accuracy: 0.1842
Epoch 9/10
24/24 [==============================] - 4s 154ms/step - loss: 0.2199 - participant_output_loss: 0.0371 - command_output_loss: 0.1827 - command_output_1_loss: 1.0659e-05 - participant_output_1_loss: 1.5979e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9993

Epoch 1/10
24/24 [==============================] - 4s 172ms/step - loss: 0.2728 - participant_output_loss: 0.1103 - command_output_loss: 0.1625 - command_output_1_loss: 2.5638e-05 - participant_output_1_loss: 2.6295e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9993 - command_output_1_accuracy: 0.1592 - participant_output_1_accuracy: 0.1880 - val_loss: 0.3377 - val_participant_output_loss: 0.1408 - val_command_output_loss: 0.1969 - val_command_output_1_loss: 1.8733e-05 - val_participant_output_1_loss: 2.9919e-05 - val_participant_output_accuracy: 0.9871 - val_command_output_accuracy: 0.9890 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.2173
Epoch 2/10
24/24 [==============================] - 3s 125ms/step - loss: 0.2247 - participant_output_loss: 0.0921 - command_output_loss: 0.1325 - command_output_1_loss: 1.9374e-05 - participant_output_1_loss: 2.4177e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.

...

Epoch 1/10
24/24 [==============================] - 5s 212ms/step - loss: 3.2493 - participant_output_loss: 1.2595 - command_output_loss: 1.9839 - command_output_1_loss: 3.1476e-04 - participant_output_1_loss: 0.0056 - participant_output_accuracy: 0.6114 - command_output_accuracy: 0.5043 - command_output_1_accuracy: 0.0903 - participant_output_1_accuracy: 0.3405 - val_loss: 2.6710 - val_participant_output_loss: 0.9303 - val_command_output_loss: 1.7404 - val_command_output_1_loss: 1.3416e-04 - val_participant_output_1_loss: 1.0160e-04 - val_participant_output_accuracy: 0.7993 - val_command_output_accuracy: 0.7716 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.0129
Epoch 2/10
24/24 [==============================] - 4s 165ms/step - loss: 2.3587 - participant_output_loss: 0.7777 - command_output_loss: 1.5807 - command_output_1_loss: 1.1870e-04 - participant_output_1_loss: 1.2188e-04 - participant_output_accuracy: 0.8769 - command_output_accuracy: 0.8602

Epoch 3/10
24/24 [==============================] - 3s 145ms/step - loss: 0.1733 - participant_output_loss: 0.0581 - command_output_loss: 0.1152 - command_output_1_loss: 1.3100e-05 - participant_output_1_loss: 3.2686e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9993 - command_output_1_accuracy: 0.0161 - participant_output_1_accuracy: 0.2221 - val_loss: 0.2546 - val_participant_output_loss: 0.0987 - val_command_output_loss: 0.1559 - val_command_output_1_loss: 8.5519e-06 - val_participant_output_1_loss: 4.1245e-05 - val_participant_output_accuracy: 0.9945 - val_command_output_accuracy: 0.9853 - val_command_output_1_accuracy: 0.0129 - val_participant_output_1_accuracy: 0.2431
Epoch 4/10
24/24 [==============================] - 3s 145ms/step - loss: 0.1503 - participant_output_loss: 0.0527 - command_output_loss: 0.0975 - command_output_1_loss: 1.2785e-05 - participant_output_1_loss: 3.0095e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9993

Epoch 7/10
24/24 [==============================] - 3s 117ms/step - loss: 0.7777 - participant_output_loss: 0.2154 - command_output_loss: 0.5623 - command_output_1_loss: 4.5034e-05 - participant_output_1_loss: 2.8243e-05 - participant_output_accuracy: 0.9920 - command_output_accuracy: 0.9953 - command_output_1_accuracy: 0.1672 - participant_output_1_accuracy: 0.1171 - val_loss: 0.8022 - val_participant_output_loss: 0.2477 - val_command_output_loss: 0.5544 - val_command_output_1_loss: 4.2629e-05 - val_participant_output_1_loss: 3.2310e-05 - val_participant_output_accuracy: 0.9779 - val_command_output_accuracy: 0.9797 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1989
Epoch 8/10
24/24 [==============================] - 3s 142ms/step - loss: 0.6212 - participant_output_loss: 0.1829 - command_output_loss: 0.4382 - command_output_1_loss: 4.1881e-05 - participant_output_1_loss: 2.7903e-05 - participant_output_accuracy: 0.9953 - command_output_accuracy: 0.

Epoch 9/10
24/24 [==============================] - 3s 119ms/step - loss: 0.0955 - participant_output_loss: 0.0481 - command_output_loss: 0.0473 - command_output_1_loss: 4.8062e-06 - participant_output_1_loss: 1.1951e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9993 - command_output_1_accuracy: 0.0462 - participant_output_1_accuracy: 0.1612 - val_loss: 0.1811 - val_participant_output_loss: 0.0901 - val_command_output_loss: 0.0910 - val_command_output_1_loss: 3.2504e-06 - val_participant_output_1_loss: 1.9710e-05 - val_participant_output_accuracy: 0.9853 - val_command_output_accuracy: 0.9890 - val_command_output_1_accuracy: 0.1694 - val_participant_output_1_accuracy: 0.1308
Epoch 10/10
24/24 [==============================] - 3s 117ms/step - loss: 0.0859 - participant_output_loss: 0.0441 - command_output_loss: 0.0418 - command_output_1_loss: 3.3120e-06 - participant_output_1_loss: 1.0922e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.999

Epoch 1/10
24/24 [==============================] - 4s 167ms/step - loss: 0.4160 - participant_output_loss: 0.0655 - command_output_loss: 0.3505 - command_output_1_loss: 4.4850e-05 - participant_output_1_loss: 2.3884e-05 - participant_output_accuracy: 0.9993 - command_output_accuracy: 0.9993 - command_output_1_accuracy: 0.4007 - participant_output_1_accuracy: 0.2033 - val_loss: 0.4677 - val_participant_output_loss: 0.1037 - val_command_output_loss: 0.3640 - val_command_output_1_loss: 2.3908e-05 - val_participant_output_1_loss: 2.9432e-05 - val_participant_output_accuracy: 0.9853 - val_command_output_accuracy: 0.9908 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.3168
Epoch 2/10
24/24 [==============================] - 3s 120ms/step - loss: 0.3391 - participant_output_loss: 0.0540 - command_output_loss: 0.2851 - command_output_1_loss: 2.4974e-05 - participant_output_1_loss: 2.3209e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.

In [27]:
resnet_model = tf.keras.models.load_model('Initial_group_model_60p_0608.h5')
predictions = resnet_model.predict(Test_Inputs)[0]
predicted_classes = np.argmax(predictions, axis=1)+1
acc_c = round(sum(x == y for x, y in zip(test_unit_participant_class, predicted_classes)) / len(test_unit_participant_class),4)

predictions = resnet_model.predict(Test_Inputs)[1]
predicted_classes = np.argmax(predictions, axis=1)
acc_s = round(sum(x == y for x, y in zip(test_unit_command_class, predicted_classes)) / len(test_unit_command_class),4)
overall_acc = acc_c + acc_s
print(acc_c,acc_s)

0.9945 0.989


In [28]:
Perfomance

,Model,Size,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8
0,Group,20%,47.540156,0.9724,0.9484,0.991,0.9550,0.9259,0.8796,0.9652,0.9565,0.9817,0.9633,1.00,0.99,0.67,0.528,0.3366
1,Group,20%,48.307249,0.9779,0.9521,1.000,0.9369,0.9352,0.8796,0.9739,0.9652,0.9908,0.9908,0.99,0.99,0.59,0.528,0.2970
2,Group,20%,46.201615,0.9613,0.9448,0.991,0.9369,0.9074,0.8611,0.9826,0.9652,0.9358,0.9725,0.99,0.99,0.67,0.432,0.2673
3,Group,20%,43.692362,0.9650,0.9521,1.000,0.9459,0.9074,0.8704,0.9826,0.9652,0.9358,0.9908,1.00,0.99,0.62,0.504,0.2673
4,Group,20%,44.504580,0.9687,0.9484,0.991,0.9550,0.9259,0.8796,0.9739,0.9652,0.9541,0.9541,1.00,0.99,0.59,0.472,0.3267
5,Group,20%,44.229166,0.9705,0.9521,1.000,0.9550,0.9074,0.8796,0.9826,0.9739,0.9633,0.9633,1.00,0.99,0.62,0.456,0.3069
6,Group,20%,44.535541,0.9632,0.9540,0.991,0.9369,0.9259,0.8981,0.9565,0.9652,0.9541,0.9817,0.99,0.99,0.67,0.496,0.3267
7,Group,20%,42.948731,0.9724,0.9595,1.000,0.9550,0.9074,0.8889,0.9913,0.9739,0.9633,0.9908,1.00,0.99,0.67,0.496,0.2574
8,Group,20%,45.444911,0.9724,0.9503,0.991,0.9459,0.9167,0.8796,0.9913,0.9652,0.9633,0.9725,1.00,0.99,0.60,0.552,0.2673
9,Group,20%,42.707019,0.9669,0.9540,0.991,0.9459,0.9167,0.8704,0.9739,0.9652,0.9541,0.9908,1.00,1.00,0.66,0.480,0.4059


In [29]:
Perfomance.to_csv('Performance_0608_training_Data_Size.csv',index=False)

8